In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
import os
import re
import ruamel.yaml
import shutil


In [16]:
yaml = ruamel.yaml.YAML()
yaml.representer.ignore_aliases = lambda *data: True

DefaultConfigPath = "Config.yaml"

with open(DefaultConfigPath, encoding="utf-8") as inyaml:
    yaml_data = yaml.load(inyaml)

In [21]:
patients_T1 = set()
patients_T2 = set()
patients_MRA = set()
for root, dirs, files in os.walk("ixi_slice"):
    for Type in yaml_data['modalities']:
        if Type in root:
            for name in files:
                if not name.startswith('.'):
                    name = re.sub(r's_20.png$', "", name)
                    name = re.sub(r'^[a-zA-Z]*-', "", name)
                    matchObj = re.search( r'\w{2}$', name)
                    name = re.sub(r'-\w*$', "", name)

                    if Type == 'T1':
                        patients_T1.add(name)
                    if Type == 'T2':
                        patients_T2.add(name)
                    if Type == 'MRA':
                        patients_MRA.add(name)
                        
patients = patients_T1 & patients_T2 & patients_MRA

In [22]:
patients

{'IXI002-Guys-0828',
 'IXI012-HH-1211',
 'IXI013-HH-1212',
 'IXI015-HH-1258',
 'IXI016-Guys-0697',
 'IXI017-Guys-0698',
 'IXI019-Guys-0702',
 'IXI020-Guys-0700',
 'IXI021-Guys-0703',
 'IXI022-Guys-0701',
 'IXI023-Guys-0699',
 'IXI024-Guys-0705',
 'IXI025-Guys-0852',
 'IXI026-Guys-0696',
 'IXI027-Guys-0710',
 'IXI028-Guys-1038',
 'IXI029-Guys-0829',
 'IXI030-Guys-0708',
 'IXI031-Guys-0797',
 'IXI033-HH-1259',
 'IXI034-HH-1260',
 'IXI035-IOP-0873',
 'IXI036-Guys-0736',
 'IXI037-Guys-0704',
 'IXI038-Guys-0729',
 'IXI039-HH-1261',
 'IXI040-Guys-0724',
 'IXI041-Guys-0706',
 'IXI042-Guys-0725',
 'IXI043-Guys-0714',
 'IXI044-Guys-0712',
 'IXI045-Guys-0713',
 'IXI046-Guys-0824',
 'IXI048-HH-1326',
 'IXI049-HH-1358',
 'IXI050-Guys-0711',
 'IXI051-HH-1328',
 'IXI052-HH-1343',
 'IXI053-Guys-0727',
 'IXI054-Guys-0707',
 'IXI055-Guys-0730',
 'IXI056-HH-1327',
 'IXI057-HH-1342',
 'IXI058-Guys-0726',
 'IXI059-HH-1284',
 'IXI060-Guys-0709',
 'IXI061-Guys-0715',
 'IXI062-Guys-0740',
 'IXI063-Guys-0742'

In [23]:
patients_train, patients_val_test = train_test_split(np.array(list(patients)), test_size=0.2)
patients_val, patients_test = train_test_split(patients_val_test, test_size=0.5)

In [25]:
for root, dirs, files in os.walk('ixi_slice'):
    for name in files:
        if not name.startswith('.'):
            name_temp = name

            name = re.sub(r's_20.png$', "", name)
            name = re.sub(r'^[a-zA-Z]*-', "", name)
            name = re.sub(r'-\w*$', "", name)
            
            if name in patients_train:
                path = os.path.join(root.replace('ixi_slice','data/train'))
                if not os.path.exists(path):
                    os.makedirs(path)
                shutil.copy(os.path.join(root,name_temp), os.path.join(path,name_temp))
            
            if name in patients_val:
                path = os.path.join(root.replace('ixi_slice','data/val'))
                if not os.path.exists(path):
                    os.makedirs(path)
                shutil.copy(os.path.join(root,name_temp), os.path.join(path,name_temp))
            
            if name in patients_test:
                path = os.path.join(root.replace('ixi_slice','data/test'))
                if not os.path.exists(path):
                    os.makedirs(path)
                shutil.copy(os.path.join(root,name_temp), os.path.join(path,name_temp))